In [ ]:
import os, sys, uproot
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import HomeMade as hm
from importlib import reload  
hm=reload(hm)
import seaborn as sb
from iminuit import Minuit
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
import shap
import dill
from AppStatFunctions import Chi2Regression,UnbinnedLH, BinnedLH, add_text_to_ax, nice_string_output
from sklearn.preprocessing import RobustScaler
from hep_ml.reweight import GBReweighter

In [ ]:
Nmax=2.4e6
uncorrelated=['v0_chi2', 'v0_px1', 'v0_phi1', 'v0_py1', 'v0_py', 'v0_py2', 'v0_phi2', 'v0_px2', 'v0_px', 'cosTheta', 'a0xy', 'a0', 'v0_y', 'v0_x', 'v0_rxyErr', 'v0_rxy','v0_z', 'pv0_z', 'pv0_y', 'ntrk_pv0', 'pv0_x']

In [ ]:
ml1=['v0_chi2', 'v0_px1', 'v0_phi1', 'v0_py1', 'v0_py', 'v0_py2', 'v0_phi2', 'v0_px2', 'v0_px']
ml2=['cosTheta', 'a0xy', 'a0', 'v0_y', 'v0_x', 'v0_rxyErr', 'v0_rxy','v0_z', 'pv0_z', 'pv0_y', 'ntrk_pv0', 'pv0_x']

In [ ]:
N = Nmax
if N>Nmax:
    N=Nmax
    print('Maxed out')

all_features = np.hstack((['v0_ks_mass'], uncorrelated))

path = "../../data/data15_13TeV.00267358.physics_MinBias.30062015_v0_per_0.root"
file = uproot.open(path)
data = file['tree'].pandas.df(all_features, entrystop = N)

path = "../../data/mc15_13TeV.361203.Pythia8_A2_MSTW2008LO_ND_minbias.30062015_v0_per_0.root"
file = uproot.open(path)
mc = file['tree'].pandas.df(np.hstack((all_features ,['trueKs'])), entrystop = N)

data = data.loc[(data.v0_ks_mass > 400) & (data.v0_ks_mass < 600)]
train_data = data.sample(frac=0.7)
test_data = data[~data.index.isin(train_data.index)]

mc = mc.loc[(mc.v0_ks_mass > 400) & (mc.v0_ks_mass < 600)]
train_mc = mc.sample(frac=0.7)
test_mc = mc[~mc.index.isin(train_mc.index)]

In [ ]:

from hep_ml.metrics_utils import ks_2samp_weighted

hist_settings = {'bins': 100, 'density': True, 'alpha': 0.7}

def draw_distributions(original, target, new_original_weights, plot=True, verbose=True):
    s=[]
    plt.figure(figsize=[20, 20])
    for id, column in enumerate(original.columns, 1):
        print(column)
        xlim = np.percentile(np.hstack([target[column]]), [0.01, 99.99])
        if plot:
            plt.subplot(5, 5, id)
            plt.hist(original[column], weights=new_original_weights, range=xlim, **hist_settings)
            plt.hist(target[column], range=xlim, **hist_settings)
            plt.title(column)
        kstest=ks_2samp_weighted(original[column], target[column], 
                                         weights1=new_original_weights, weights2=np.ones(len(target), dtype=float))
        if verbose:
            print('KS over ', column, ' = ',kstest)
        s.append(kstest)
    return np.sum(s)/len(s), np.array(s)

In [ ]:
def compare_distributions(original, target, new_original_weights):
    s=[]
    for id, column in enumerate(original.columns, 1):
        s.append(ks_2samp_weighted(original[column], target[column], 
                                         weights1=new_original_weights, weights2=np.ones(len(target), dtype=float)))
    return np.sum(s)/len(s), s

In [ ]:
def scale_split(mcs, datas, split, paramlist, no_transform):
    mc1, data1=mcs[paramlist], datas[paramlist]
    testlen=int(len(mc)*(1-split))
    mct, datat=RobustScaler().fit(mc1), RobustScaler().fit(mc1)
    mc1, data1=mct.transform(mc1), datat.transform(data1)
    mc1, data1=pd.DataFrame(mc1, columns=paramlist), pd.DataFrame(data1, columns=paramlist)
    for no in no_transform:
        mc1[no]=mcs[no].to_numpy()
        data1[no]=datas[no].to_numpy()
    true=mc.trueKs
    return mc1[:testlen], mc1[testlen:], data1[:testlen], data1[testlen:], true[:testlen], true[testlen:]

In [ ]:
mc_test, mc_train, data_test, data_train, truetest, truetrain=scale_split(mc, data, 0.7, uncorrelated, ['v0_chi2', 'cosTheta', 'v0_ks_mass'])
mcweights_train = np.ones(len(mc_train))
mcweights_test = np.ones(len(mc_test))

Checking out the scaled stuff

In [ ]:
s, sarray=draw_distributions(mc_test[uncorrelated], data_test[uncorrelated], mcweights_test)

Checking out the non_scaled population

In [ ]:
sorig, sorigarray=draw_distributions(mc[uncorrelated], data[uncorrelated], np.ones(len(mc)))

In [ ]:
sorig/s

In [ ]:
sorig

### We gain a bit with the new scaled and centereed distributions, about 3%

In [ ]:
paramsure=['v0_ks_mass','v0_chi2', 'v0_px1', 'v0_phi1', 'v0_py1', 'v0_py', 'v0_py2', 'v0_phi2', 'v0_px2', 'v0_px', 'cosTheta', 'a0xy', 'v0_x', 'v0_rxyErr', 'v0_z' ]
extra=[ 'a0xy', 'a0,' 'v0_x', 'v0_rxyErr', 'v0_z', 'pv0_z']
extra=[ 'a0xy', 'v0_rxyErr']

In [ ]:
features=paramsure
slist=[]
#trying to add ks mass to non scaled
mc_test, mc_train, data_test, data_train, truetest, truetrain=scale_split(mc, data, 0.7, features, ['v0_chi2', 'cosTheta', 'v0_ks_mass'])
reweighter = GBReweighter(n_estimators=50, learning_rate=0.1, max_depth=5, min_samples_leaf=1000)
reweighter.fit(mc_train[features], data_train[features])

In [ ]:
gb_weights_test = reweighter.predict_weights(mc_test[features])
# validate reweighting rule on the test part comparing 1d projections
s, _ =draw_distributions(mc_test[features], data_test[features], gb_weights_test, verbose=0)
slist.append(s)

In [ ]:
s, sorig/s

## 6 times better! Watch out for disturbance in mass though

In [ ]:
#NAMES: mc_test, mc_train, data_test, data_train, truetest, truetrain
params=features[1:]
import xgboost
train_weights = reweighter.predict_weights(mc_train[paramsure])
test_weights = reweighter.predict_weights(mc_test[paramsure])
eval_s = [(mc_train[params], truetrain), (mc_test[params], truetest)]
eval_weights=[train_weights, test_weights]
model = xgboost.XGBClassifier(learning_rate = 0.02,n_estimators = 1000)
model.fit(mc_train[params], truetrain, sample_weight=train_weights, verbose=True,eval_set=eval_s, sample_weight_eval_set=eval_weights,
          early_stopping_rounds=30,eval_metric ="auc")

In [ ]:
# import shap

# # load JS visualization code to notebook
# shap.initjs()

# # explain the model's predictions using SHAP
# # (same syntax works for LightGBM, CatBoost, scikit-learn and spark models)
# explainer = shap.TreeExplainer(model)
# shap_values = explainer.shap_values(mc_train[params])

# # visualize the first prediction's explanation (use matplotlib=True to avoid Javascript)
# shap.summary_plot(shap_values, mc_train[params])

In [ ]:
pxg=model.predict_proba(data_train[params])[:,1]
plt.hist(pxg, bins=100);
# plt.yscale('log')

In [ ]:
true=pxg>0.7
plt.hist(data_train.v0_ks_mass[true], bins=100);

In [ ]:
#just for fun
pxgt=model.predict_proba(data_test[params])[:,1]
truetest=pxgt>0.7
plt.hist(data_test.v0_ks_mass[truetest], bins=100);

In [ ]:
eval_s = [(data_train[params], true), (data_test[params], truetest)]
# tsts.append(X_train)
modeldata = xgboost.XGBClassifier(learning_rate = 0.02,n_estimators = 200)
modeldata.fit(data_train[params], true,verbose=True,eval_set=eval_s,early_stopping_rounds=30,eval_metric ="auc")

In [ ]:
p_final=modeldata.predict_proba(data_test[params])[:,0]
true_final=p_final>0.75
plt.hist(data_test.v0_ks_mass[true_final], bins=100);

In [ ]:
plt.hist(data_test.v0_ks_mass[~true_final], bins=100);

In [ ]:
fig1, ax1=plt.subplots(figsize=(10,8))
hm.roc_curve_data(data_test.v0_ks_mass, p_final, Npoints = 100, bins = 100, range = (400, 600), ax_roc = ax1 ,\
             verbose = True, plimit = 0.01)

In [ ]:
mc_test, mc_train, data_test, data_train, truetest, truetrain=scale_split(mc, data, 0.7, np.hstack((uncorrelated, ['v0_ks_mass'])), ['v0_chi2','v0_ks_mass', 'cosTheta'])
eval_s = [(mc_train[uncorrelated], truetrain), (mc_test[uncorrelated], truetest)]
model = xgboost.XGBClassifier(learning_rate = 0.02,n_estimators = 200)
model.fit(mc_train[uncorrelated], truetrain, verbose=True,eval_set=eval_s,
          early_stopping_rounds=30,eval_metric ="auc")

In [ ]:
pxg=model.predict_proba(data_train[uncorrelated])[:,1]
plt.hist(pxg, bins=100);
# plt.yscale('log')
true=pxg>0.7
plt.hist(data_train.v0_ks_mass[true], bins=100);
#just for fun
#just for fun
pxgt=model.predict_proba(data_test[uncorrelated])[:,1]
true_test=pxgt>0.7
plt.hist(data_test.v0_ks_mass[true_test], bins=100);


eval_s = [(data_train[uncorrelated], true), (data_test[uncorrelated], true_test)]
modeldata = xgboost.XGBClassifier(learning_rate = 0.02,n_estimators = 200)
modeldata.fit(data_train[uncorrelated], true,verbose=True,eval_set=eval_s,early_stopping_rounds=30,eval_metric ="auc")

In [ ]:
fig1, ax1=plt.subplots(figsize=(10,8))
hm.roc_curve_data(data_test.v0_ks_mass, p_final, Npoints = 100, bins = 100, range = (400, 600), ax_roc = ax1 ,\
             verbose = True, plimit = 0.01)

In [ ]:
p_final=modeldata.predict_proba(data_test[uncorrelated])[:,0]
true_final=p_final>0.8
plt.hist(data_test.v0_ks_mass[true_final], bins=100);

# Testing straight training in MC

In [ ]:
train_weights = reweighter.predict_weights(mc_train[paramsure])
test_weights = reweighter.predict_weights(mc_test[paramsure])
eval_s = [(mc_train[params], truetrain), (mc_test[params], truetest)]
eval_weights=[train_weights, test_weights]
modelw = xgboost.XGBClassifier(learning_rate = 0.02,n_estimators = 200)
modelw.fit(mc_train[params], truetrain, sample_weight=train_weights, verbose=True,eval_set=eval_s, sample_weight_eval_set=eval_weights,
          early_stopping_rounds=30,eval_metric ="auc")

In [ ]:
pxgw=modelw.predict_proba(data_train[params])[:,0]

In [ ]:
fig1, ax1=plt.subplots(figsize=(10,8))
hm.roc_curve_data(data_train.v0_ks_mass, pxgw, Npoints = 100, bins = 100, range = (400, 600), ax_roc = ax1 ,\
             verbose = False, plimit = 0.01)

In [ ]:
mc_test, mc_train, data_test, data_train, truetest, truetrain=scale_split(mc, data, 0.7, np.hstack((uncorrelated, ['v0_ks_mass'])), ['v0_chi2','v0_ks_mass', 'cosTheta'])
eval_s = [(mc_train[uncorrelated], truetrain), (mc_test[uncorrelated], truetest)]
modelnw = xgboost.XGBClassifier(learning_rate = 0.02,n_estimators = 200)
modelnw.fit(mc_train[uncorrelated], truetrain, eval_set=eval_s, verbose=True, early_stopping_rounds=30,eval_metric ="auc")

In [ ]:
pxgnw=modelnw.predict_proba(data_train[uncorrelated])[:,0]

In [ ]:
fig1, ax1=plt.subplots(figsize=(10,8))
a,b=hm.roc_curve_data(data_train.v0_ks_mass, pxgnw, Npoints = 50, bins = 100, range = (400, 600), ax_roc = ax1 ,\
             verbose = False, plimit = 0.9)

In [ ]:
ax1.set(xlabel='FPR', ylabel='TPR', xlim=(-0.05,1.05), ylim=(-0.05,1.05), title=f'Estimated ROC curve, AUC $\approx$ {1+a}')
plt.savefig('roccurvenoreweigh.pdf')

In [ ]:
uncorrelated=params
mc_test, mc_train, data_test, data_train, truetest, truetrain=scale_split(mc, data, 0.7, np.hstack((uncorrelated, ['v0_ks_mass'])), ['v0_chi2','v0_ks_mass', 'cosTheta'])
eval_s = [(pd.concat((data_train[uncorrelated], mc_train[uncorrelated])), np.hstack((np.ones(len(data_train)), np.zeros(len(mc_train))))), 
          (pd.concat((data_test[uncorrelated], mc_test[uncorrelated])),  np.hstack((np.ones(len(data_test)), np.zeros(len(mc_test)))))]
modelnw = xgboost.XGBClassifier(learning_rate = 0.02,n_estimators = 200)
modelnw.fit(pd.concat((data_train[uncorrelated], mc_train[uncorrelated])), np.hstack((np.ones(len(data_train)), np.zeros(len(mc_train)))),
            eval_set=eval_s, verbose=True, early_stopping_rounds=30,eval_metric ="auc")

In [ ]:
pnw=modelnw.predict_proba(pd.concat((data_test[uncorrelated], mc_test[uncorrelated])))[:,1]
true=pnw>0.7
plt.hist(pnw, bins=100);

In [ ]:
train_weights = reweighter.predict_weights(mc_train[paramsure])
test_weights = reweighter.predict_weights(mc_test[paramsure])
eval_weights=[np.hstack((np.ones(len(data_train)), train_weights)), np.hstack((np.ones(len(data_test)), test_weights))]

In [ ]:
mc_test, mc_train, data_test, data_train, truetest, truetrain=scale_split(mc, data, 0.7, np.hstack((uncorrelated, ['v0_ks_mass'])), ['v0_chi2','v0_ks_mass', 'cosTheta'])
eval_s = [(pd.concat((data_train[params], mc_train[params])), np.hstack((np.ones(len(data_train)), np.zeros(len(mc_train))))), 
          (pd.concat((data_test[params], mc_test[params])),  np.hstack((np.ones(len(data_test)), np.zeros(len(mc_test)))))]
modelw = xgboost.XGBClassifier(learning_rate = 0.02,n_estimators = 200)
modelw.fit(pd.concat((data_train[params], mc_train[params])), np.hstack((np.ones(len(data_train)), np.zeros(len(mc_train)))), 
            sample_weight=eval_weights[0], eval_set=eval_s, sample_weight_eval_set=eval_weights,
            verbose=True, early_stopping_rounds=30,eval_metric ="auc")

In [ ]:
pw=modelw.predict_proba(pd.concat((data_test[params], mc_test[params])))[:,1]
true=pw>0.8
plt.hist(pw, bins=100);

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score
fpr, tpr, thresholds = roc_curve(y_true=np.hstack((np.ones(len(data_test)), np.zeros(len(mc_test)))),
                                y_score=pw)

fig, ax = plt.subplots()
ax.plot(tpr,fpr)
ax.plot([0,1],[0,1],c='grey',linestyle='--')
ax.set_title(f'ROC curve, AUC: {1-roc_auc_score(y_true=np.hstack((np.ones(len(data_test)), np.zeros(len(mc_test)))),y_score=pw)}')

In [ ]:
import shap

# load JS visualization code to notebook
shap.initjs()

# explain the model's predictions using SHAP
# (same syntax works for LightGBM, CatBoost, scikit-learn and spark models)
explainer = shap.TreeExplainer(modelw)
shap_values = explainer.shap_values(pd.concat((data_test[params], mc_test[params])))

# visualize the first prediction's explanation (use matplotlib=True to avoid Javascript)
shap.summary_plot(shap_values, pd.concat((data_test[params], mc_test[params])))

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score
fpr, tpr, thresholds = roc_curve(y_true=np.hstack((np.ones(len(data_test)), np.zeros(len(mc_test)))),
                                y_score=pnw)

fig, ax = plt.subplots()
ax.plot(tpr,fpr)
ax.plot([0,1],[0,1],c='grey',linestyle='--')
ax.set_title(f'ROC curve, AUC: {1-roc_auc_score(y_true=np.hstack((np.ones(len(data_test)), np.zeros(len(mc_test)))),y_score=pnw)}')

In [ ]:
import shap

# load JS visualization code to notebook
shap.initjs()

# explain the model's predictions using SHAP
# (same syntax works for LightGBM, CatBoost, scikit-learn and spark models)
explainer = shap.TreeExplainer(modelnw)
shap_values = explainer.shap_values(pd.concat((data_test[params], mc_test[params])))

# visualize the first prediction's explanation (use matplotlib=True to avoid Javascript)
shap.summary_plot(shap_values, pd.concat((data_test[params], mc_test[params])))

In [ ]:
params

# Let's try a different scaling method: MinMaxScaling (with robustness)

In [ ]:
so, solist=draw_distributions(mc[all_features], data[all_features], np.ones(len(mc)))

In [ ]:
from sklearn.preprocessing import MinMaxScaler
def scale_split2(mcs, datas, split, quantile, paramlist, no_transform):
    mc1, data1=mcs[paramlist], datas[paramlist]
    testlen=int(len(mc)*(1-split))
    
    for param in paramlist:
        if param not in no_transform:
            print(param+' is being transformed' )
            q_mc, q_data = (mcs.quantile(1-quantile)[param], mcs.quantile(quantile)[param]), (datas.quantile(1-quantile)[param], datas.quantile(quantile)[param])
            mcq, dataq = mcs.loc[(mcs[param] > q_mc[0]) & (mcs[param] < q_mc[1])], datas.loc[(datas[param] > q_data[0]) & (datas[param] < q_data[1])]
            mcscaler, datascaler=MinMaxScaler().fit( (mcq[param]).to_numpy().reshape(-1, 1) ), MinMaxScaler().fit( (dataq[param]).to_numpy().reshape(-1, 1))  
            mcscaled, datascaled= mcscaler.transform( (mcs[param]).to_numpy().reshape(-1, 1) ), datascaler.transform( (datas[param]).to_numpy().reshape(-1, 1) ) 
            mc1[param], data1[param]=mcscaled, datascaled
    true=mc.trueKs
    return mc1[:testlen], mc1[testlen:], data1[:testlen], data1[testlen:], true[:testlen], true[testlen:]

In [ ]:
uncorrelated=['v0_chi2', 'v0_px1', 'v0_phi1', 'v0_py1', 'v0_py', 'v0_py2', 'v0_phi2','v0_px2', 'v0_px', 
              'cosTheta', 'a0xy', 'a0', 'v0_y', 'v0_x', 'v0_rxyErr', 'v0_rxy','v0_z', 'pv0_z', 'pv0_y', 'ntrk_pv0', 'pv0_x']
mc_test, mc_train, data_test, data_train, truetest, truetrain=scale_split2(mc, data, 
                    0.7, 0.78, np.hstack((uncorrelated, ['v0_ks_mass'])), ['v0_chi2','v0_ks_mass', 'cosTheta'])

In [ ]:
ses=[]
for i in range(70,80):
    split=i/100
    mc_test, mc_train, data_test, data_train, truetest, truetrain=scale_split2(mc, data, 
                    0.7, split, np.hstack((uncorrelated, ['v0_ks_mass'])), ['v0_chi2','v0_ks_mass', 'cosTheta'])
    s1, s1list=compare_distributions(mc_train, data_train, np.ones(len(mc_train)))
    ses.append(s1)

In [ ]:
plt.plot(range(70,80),ses)

In [ ]:
features=np.hstack((['v0_ks_mass'], uncorrelated))
mc_test, mc_train, data_test, data_train, truetest, truetrain=scale_split2(mc, data, 
                    0.78, split, features, ['v0_chi2','v0_ks_mass', 'cosTheta'])

In [ ]:
sn, snlist=draw_distributions(mc_test, data_test, np.ones(len(mc_test)))

In [ ]:
plt.plot(np.array(solist)-np.array(snlist))
plt.hlines(0, 0,22, linestyle='dashed')
all_features

In [ ]:
so/sn

# let's do ml1/ml2 and ml1+ml2

In [ ]:
print(all_features)

In [ ]:

reweighter12 = GBReweighter(n_estimators=50, learning_rate=0.1, max_depth=5, min_samples_leaf=1000)
reweighter12.fit(mc_train[all_features], data_train[all_features])

In [ ]:
gb_weights_test = reweighter12.predict_weights(mc_test[features])
# validate reweighting rule on the test part comparing 1d projections
s12, _ =draw_distributions(mc_test[features], data_test[features], gb_weights_test, verbose=1)

In [ ]:
#need to get mass in on both

ML1, ML2=np.hstack((['v0_ks_mass'], ml1)), np.hstack((['v0_ks_mass'], ml2))

In [ ]:
reweighter1 = GBReweighter(n_estimators=50, learning_rate=0.1, max_depth=5, min_samples_leaf=1000)
reweighter1.fit(mc_train[ML1], data_train[ML1])

In [ ]:
gb_weights1 = reweighter1.predict_weights(mc_test[ML1])
# validate reweighting rule on the test part comparing 1d projections
s1, _ =draw_distributions(mc_test[ML1], data_test[ML1], gb_weights1, verbose=1)

In [ ]:
reweighter2 = GBReweighter(n_estimators=50, learning_rate=0.1, max_depth=5, min_samples_leaf=1000)
reweighter2.fit(mc_train[ML2], data_train[ML2])

In [ ]:
gb_weights2 = reweighter2.predict_weights(mc_test[ML2])
# validate reweighting rule on the test part comparing 1d projections
s2, _ =draw_distributions(mc_test[ML2], data_test[ML2], gb_weights2, verbose=1)

Alright so it's ml2 that's fucking shit up. Let's find out where.

In [ ]:
slist=[]
check=['v0_ks_mass','cosTheta','a0xy','a0','v0_y','v0_x','v0_rxyErr','v0_rxy','v0_z']
extra=['pv0_z','pv0_y','ntrk_pv0','pv0_x']
for e in extra:
    tot=np.hstack((check, e))
    rw = GBReweighter(n_estimators=50, learning_rate=0.1, max_depth=5, min_samples_leaf=1000)
    rw.fit(mc_train[tot], data_train[tot])
    weightstest = rw.predict_weights(mc_test[tot])
    # validate reweighting rule on the test part comparing 1d projections
    ss, _ =draw_distributions(mc_test[tot], data_test[tot], weightstest, verbose=0, plot=0)
    slist.append(ss)

In [ ]:
plt.plot(extra, slist)

So we can probably keep ntrk_pv0 and pv0_x but it turns out that keeping both is bad

Let's also try removing a0 since most of that information is in a0xy

In [ ]:
ML2=['v0_ks_mass','cosTheta','a0','v0_y','v0_x','v0_rxyErr','v0_rxy','v0_z','ntrk_pv0']
obsolote=[ 'pv0_x','pv0_z','pv0_x', 'a0xy']
reweighter2 = GBReweighter(n_estimators=100, learning_rate=0.1, max_depth=4, min_samples_leaf=1000)
reweighter2.fit(mc_train[ML2], data_train[ML2])

In [ ]:
gb_weights2 = reweighter2.predict_weights(mc_test[ML2])
# validate reweighting rule on the test part comparing 1d projections
s2, _ =draw_distributions(mc_test[ML2], data_test[ML2], gb_weights2, verbose=1)

Throw out a0xy and keep ntrk_pv0

Let's try the whole thing while still drawing the obsolete parameters 

In [ ]:
# validate reweighting rule on the test part comparing 1d projections
s2, _ =draw_distributions(mc_test[ml2], data_test[ml2], gb_weights2, verbose=1)

Let's get the reweighter for the whole thing

In [ ]:
train_features=['v0_ks_mass', 'v0_chi2', 'v0_px1', 'v0_phi1', 'v0_py1', 'v0_py',
       'v0_py2', 'v0_phi2', 'v0_px2', 'v0_px', 'cosTheta', 'a0',
       'v0_y', 'v0_x', 'v0_rxyErr', 'v0_rxy', 'v0_z', 'ntrk_pv0']
reweighter12 = GBReweighter(n_estimators=50, learning_rate=0.1, max_depth=5, min_samples_leaf=1000)
reweighter12.fit(mc_train[train_features], data_train[train_features])

gb_weights_test = reweighter12.predict_weights(mc_test[train_features])
# validate reweighting rule on the test part comparing 1d projections
s12, _ =draw_distributions(mc_test[train_features], data_test[train_features], gb_weights_test, verbose=1)

In [ ]:
s12, _ =draw_distributions(mc_test[all_features], data_test[all_features], gb_weights_test, verbose=1)

In [ ]:
s1, s2, s12, so

Testing whether xgboost can see the difference for the three scenarios

Let's check ML1
- can we make a classifier that can tell the difference?
- how does prediction for this do if we;

1: Do not reweigh MC and use it in data

2: Reweigh MC

In [ ]:
train_weights1 = reweighter1.predict_weights(mc_train[ML1])
test_weights1 = reweighter1.predict_weights(mc_test[ML1])
eval_weights1=[np.hstack((np.ones(len(data_train)), train_weights1)), np.hstack((np.ones(len(data_test)), test_weights1))]

In [ ]:
import xgboost
eval_s = [(pd.concat((data_train[ml1], mc_train[ml1])), np.hstack((np.ones(len(data_train)), np.zeros(len(mc_train))))), 
          (pd.concat((data_test[ml1], mc_test[ml1])),  np.hstack((np.ones(len(data_test)), np.zeros(len(mc_test)))))]
modelw1 = xgboost.XGBClassifier(learning_rate = 0.02,n_estimators = 200)
modelw1.fit(pd.concat((data_train[ml1], mc_train[ml1])), np.hstack((np.ones(len(data_train)), np.zeros(len(mc_train)))), 
            sample_weight=eval_weights1[0], eval_set=eval_s, sample_weight_eval_set=eval_weights1,
            verbose=True, early_stopping_rounds=30,eval_metric ="auc")

In [ ]:
p1=modelw1.predict_proba(pd.concat((data_test[ml1], mc_test[ml1])))[:,0]
plt.hist(p1, bins=100)
truetest=np.hstack((np.ones(len(data_test)), np.zeros(len(mc_test))))

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score
fpr, tpr, thresholds = roc_curve(y_true=truetest,
                                y_score=p1)

fig, ax = plt.subplots()
ax.plot(tpr,fpr)
ax.plot([0,1],[0,1],c='grey',linestyle='--')
ax.set_title(f'ROC curve for ML1, reweighed, AUC: {1-roc_auc_score(y_true=truetest,y_score=p1)}')

In [ ]:
features=np.hstack((['v0_ks_mass'], uncorrelated))
uncorrelated=['v0_chi2', 'v0_px1', 'v0_phi1', 'v0_py1', 'v0_py', 'v0_py2', 'v0_phi2','v0_px2', 'v0_px', 
              'cosTheta', 'a0xy', 'a0', 'v0_y', 'v0_x', 'v0_rxyErr', 'v0_rxy','v0_z', 'pv0_z', 'pv0_y', 'ntrk_pv0', 'pv0_x']
mc_test, mc_train, data_test, data_train, truetest, truetrain=scale_split2(mc, data, 
                    0.7, 0.78, np.hstack((uncorrelated, ['v0_ks_mass'])), ['v0_chi2','v0_ks_mass', 'cosTheta'])
eval_s1 = [(mc_train[ml1], truetrain), (mc_test[ml1], truetest)]
modeld1 = xgboost.XGBClassifier(learning_rate = 0.02,n_estimators = 200)
modeld1.fit(mc_train[ml1], truetrain, verbose=True,eval_set=eval_s1, early_stopping_rounds=30,eval_metric ="auc")

In [ ]:
p1_d=modeld1.predict_proba((data_train[ml1]))[:,1]
true1_d=p1_d>0.5
plt.hist(data_train.v0_ks_mass[true1_d], bins=100);

In [ ]:
fig1, ax1=plt.subplots(figsize=(10,8))
a,b=hm.roc_curve_data(data_train.v0_ks_mass, 1-p1_d, Npoints = 50, bins = 100, range = (400, 600), ax_roc = ax1 ,\
             verbose = False, plimit = 0.9)
1+a

In [ ]:
eval_weights1=[train_weights1, test_weights1]
eval_s1 = [(mc_train[ml1], truetrain), (mc_test[ml1], truetest)]
modeldw1 = xgboost.XGBClassifier(learning_rate = 0.02,n_estimators = 200)
modeldw1.fit(mc_train[ml1], truetrain, sample_weight=train_weights1, verbose=True,eval_set=eval_s1, 
             sample_weight_eval_set=eval_weights1,early_stopping_rounds=30,eval_metric ="auc")

In [ ]:
p1_dw=modeldw1.predict_proba((data_train[ml1]))[:,1]
true1_dw=p1_dw>0.5
plt.hist(p1_dw)

In [ ]:
plt.hist(data_train.v0_ks_mass[true1_dw], bins=100);

In [ ]:
fig1, ax1=plt.subplots(figsize=(10,8))
a,b=hm.roc_curve_data(data_train.v0_ks_mass, 1-p1_dw, Npoints = 50, bins = 100, range = (400, 600), ax_roc = ax1 ,\
             verbose = False, plimit = 0.9)
1+a

Let's check ML2

In [ ]:
train_weights2 = reweighter2.predict_weights(mc_train[ML2])
test_weights2 = reweighter2.predict_weights(mc_test[ML2])
eval_weights2=[np.hstack((np.ones(len(data_train)), train_weights2)), np.hstack((np.ones(len(data_test)), test_weights2))]

In [ ]:
eval_s2 = [(pd.concat((data_train[ml2], mc_train[ml2])), np.hstack((np.ones(len(data_train)), np.zeros(len(mc_train))))), 
          (pd.concat((data_test[ml2], mc_test[ml2])),  np.hstack((np.ones(len(data_test)), np.zeros(len(mc_test)))))]
modelw2 = xgboost.XGBClassifier(learning_rate = 0.02,n_estimators = 200)
modelw2.fit(pd.concat((data_train[ml2], mc_train[ml2])), np.hstack((np.ones(len(data_train)), np.zeros(len(mc_train)))), 
            sample_weight=eval_weights1[0], eval_set=eval_s2, sample_weight_eval_set=eval_weights2,
            verbose=True, early_stopping_rounds=30,eval_metric ="auc")

In [ ]:
p2=modelw2.predict_proba(pd.concat((data_test[ml2], mc_test[ml2])))[:,0]
plt.hist(p2, bins=100)
true_test=np.hstack((np.ones(len(data_test)), np.zeros(len(mc_test))))

In [ ]:
fpr, tpr, thresholds = roc_curve(y_true=true_test,
                                y_score=p2)

fig, ax = plt.subplots()
ax.plot(tpr,fpr)
ax.plot([0,1],[0,1],c='grey',linestyle='--')
ax.set_title(f'ROC curve for ML2, reweighed, AUC: {1-roc_auc_score(y_true=true_test,y_score=p2)}')

In [ ]:
eval_s2 = [(mc_train[ml2], truetrain), (mc_test[ml2], truetest)]
modeld2 = xgboost.XGBClassifier(learning_rate = 0.02,n_estimators = 200)
modeld2.fit(mc_train[ml2], truetrain, verbose=True,eval_set=eval_s2, early_stopping_rounds=30,eval_metric ="auc")

In [ ]:
p2_d=modeld2.predict_proba((data_train[ml2]))[:,1]
true2_d=p2_d>0.7
plt.hist(p2_d)

In [ ]:
plt.hist(data_train.v0_ks_mass[true2_d], bins=100);

In [ ]:
fig1, ax1=plt.subplots(figsize=(10,8))
a,b=hm.roc_curve_data(data_train.v0_ks_mass, 1-p2_d, Npoints = 50, bins = 100, range = (400, 600), ax_roc = ax1 ,\
             verbose = False, plimit = 0.9)
1+a

In [ ]:
eval_weights2=[train_weights2, test_weights2]
eval_s2 = [(mc_train[ml2], truetrain), (mc_test[ml2], truetest)]
modeldw2 = xgboost.XGBClassifier(learning_rate = 0.02,n_estimators = 200)
modeldw2.fit(mc_train[ml2], truetrain, sample_weight=train_weights2, verbose=True,eval_set=eval_s2, 
             sample_weight_eval_set=eval_weights2,early_stopping_rounds=50,eval_metric ="auc")

In [ ]:
p2_dw=modeldw2.predict_proba((data_train[ml2]))[:,1]
true2_dw=p2_dw>0.62
plt.hist(p2_dw)

In [ ]:
plt.hist(data_train.v0_ks_mass[true2_dw], bins=100);

In [ ]:
fig1, ax1=plt.subplots(figsize=(10,8))
a,b=hm.roc_curve_data(data_train.v0_ks_mass, 1-p2_dw, Npoints = 50, bins = 100, range = (400, 600), ax_roc = ax1 ,\
             verbose = False, plimit = 0.9)
1+a

So this is inconclusive as towards whether the reweighted instance is better or not

Let's go for ml1+ml2

In [ ]:
train_weights12 = reweighter12.predict_weights(mc_train[train_features])
test_weights12 = reweighter12.predict_weights(mc_test[train_features])
eval_weights12=[np.hstack((np.ones(len(data_train)), train_weights12)), np.hstack((np.ones(len(data_test)), test_weights12))]

In [ ]:
ml12=uncorrelated
eval_s12 = [(pd.concat((data_train[ml12], mc_train[ml12])), np.hstack((np.ones(len(data_train)), np.zeros(len(mc_train))))), 
          (pd.concat((data_test[ml12], mc_test[ml12])),  np.hstack((np.ones(len(data_test)), np.zeros(len(mc_test)))))]
modelw12 = xgboost.XGBClassifier(learning_rate = 0.02,n_estimators = 200)
modelw12.fit(pd.concat((data_train[ml12], mc_train[ml12])), np.hstack((np.ones(len(data_train)), np.zeros(len(mc_train)))), 
            sample_weight=eval_weights12[0], eval_set=eval_s12, sample_weight_eval_set=eval_weights12,
            verbose=True, early_stopping_rounds=30,eval_metric ="auc")

In [ ]:
p12=modelw12.predict_proba(pd.concat((data_test[ml12], mc_test[ml12])))[:,0]
plt.hist(p12, bins=100)
true_test=np.hstack((np.ones(len(data_test)), np.zeros(len(mc_test))))

In [ ]:
fpr, tpr, thresholds = roc_curve(y_true=true_test,
                                y_score=p12)

fig, ax = plt.subplots()
ax.plot(tpr,fpr)
ax.plot([0,1],[0,1],c='grey',linestyle='--')
ax.set_title(f'ROC curve for ML1+ML2, reweighed, AUC: {1-roc_auc_score(y_true=true_test,y_score=p12)}')

In [ ]:
eval_s12 = [(mc_train[ml12], truetrain), (mc_test[ml12], truetest)]
modeld12 = xgboost.XGBClassifier(learning_rate = 0.02,n_estimators = 200)
modeld12.fit(mc_train[ml12], truetrain, verbose=True,eval_set=eval_s12, early_stopping_rounds=30,eval_metric ="auc")

In [ ]:
p12_d=modeld12.predict_proba((data_train[ml12]))[:,1]
true12_d=p12_d>0.75
plt.hist(p12_d, bins=50)

In [ ]:
plt.hist(data_train.v0_ks_mass[true12_d], bins=100);

In [ ]:
fig1, ax1=plt.subplots(figsize=(10,8))
a,b=hm.roc_curve_data(data_train.v0_ks_mass, 1-p12_d, Npoints = 50, bins = 100, range = (400, 600), ax_roc = ax1 ,\
             verbose = False, plimit = 0.9)
1+a

In [ ]:
eval_weights12=[train_weights12, test_weights12]
eval_s12 = [(mc_train[ml12], truetrain), (mc_test[ml12], truetest)]
modeldw12 = xgboost.XGBClassifier(learning_rate = 0.02,n_estimators = 200)
modeldw12.fit(mc_train[ml12], truetrain, sample_weight=train_weights12, verbose=True,eval_set=eval_s12, 
             sample_weight_eval_set=eval_weights12,early_stopping_rounds=50,eval_metric ="auc")

In [ ]:
p12_dw=modeldw12.predict_proba((data_train[ml12]))[:,1]
true12_dw=p12_dw>0.75
plt.hist(p12_dw, bins=50)

In [ ]:
plt.hist(data_train.v0_ks_mass[true12_dw], bins=100);

In [ ]:
fig1, ax1=plt.subplots(figsize=(10,8))
a,b=hm.roc_curve_data(data_train.v0_ks_mass, 1-p12_dw, Npoints = 50, bins = 100, range = (400, 600), ax_roc = ax1 ,\
             verbose = False, plimit = 0.9)
1+a